# AKLT Forced Measurement Protocol

This notebook demonstrates forced measurement on spin-1 (S=1) chains using:

- **Protocol A**: `SpinSectorProjection` (coherent projection, preserves superposition)

## Physics Background

Coherent projection removes unwanted spin sectors (S=2) while preserving quantum superposition of allowed sectors (S=0 and S=1). This enables deterministic convergence to the AKLT ground state.

## Background: AKLT State

The **AKLT state** (Affleck-Kennedy-Lieb-Tasaki) is the unique ground state of a spin-1 chain with the Hamiltonian:

$$H = \sum_i \left[ \vec{S}_i \cdot \vec{S}_{i+1} + \frac{1}{3} (\vec{S}_i \cdot \vec{S}_{i+1})^2 \right]$$

where $\vec{S}_i$ are spin-1 operators.

### Total Spin Sectors

Two spin-1 particles can combine into three total spin sectors:
- **S = 0** (singlet, 1 state)
- **S = 1** (triplet, 3 states)
- **S = 2** (quintet, 5 states)

The AKLT state lives in the S ∈ {0, 1} subspace—it contains no S=2 quintets.

### String Order Parameter

The string order parameter is a non-local observable that characterizes the AKLT ground state:

$$O_{\text{string}}(i, j) = \langle S^z_i \exp\left(i\pi \sum_{k=i+1}^{j-1} S^z_k\right) S^z_j \rangle$$

For the AKLT ground state on a periodic chain:

$$|O_{\text{string}}| \to \frac{4}{9} \approx 0.444$$

This is our validation metric: if the protocol converges to AKLT, we should see |SO| ≈ 4/9.

## Setup

Activate the project and load required packages:

In [1]:
using Pkg; Pkg.activate(dirname(@__DIR__))
using QuantumCircuitsMPS
using LinearAlgebra
using ITensorMPS
using Printf

  Activating project at `/mnt/d/Rutgers/QuantumCircuitsMPS.jl`


## Parameters

Define the system parameters for both protocols:

In [2]:
L = 8             # Chain length (divisible by 4 for NNN coverage)
n_layers = L       # Number of projection layers
bc = :periodic     # Boundary conditions (less ambiguous than :open)
p_nn = 0.0         # Probability of NN projection
maxdim = 128       # Maximum bond dimension

# NOTE on boundary conditions:
# We use bc=:periodic which gives well-defined physics for AKLT:
# - NN AKLT: |SO| ≈ 4/9, S = 2 (in base 2)
# - NNN AKLT: |SO| ≈ (4/9)² ≈ 0.198, S = 4 (in base 2)

println("System Parameters:")
println("  L = $L (chain length)")
println("  n_layers = $n_layers")
println("  bc = $bc")
println("  p_nn = $p_nn (probability of NN projection)")
println("  p_nnn = $(1-p_nn) (probability of NNN projection)")
println("  maxdim = $maxdim")
println()

System Parameters:
  L = 8 (chain length)
  n_layers = 8
  bc = periodic
  p_nn = 0.0 (probability of NN projection)
  p_nnn = 1.0 (probability of NNN projection)
  maxdim = 128



## Physics Sanity Check

Expected values for bc=:periodic with base 2 logarithm:

| p_nn | p_nnn | Ground State | |SO|           | S (von Neumann) |
|------|-------|--------------|----------------|-----------------||
|  1   |   0   | NN AKLT      | 4/9 ≈ 0.444    |        2        |
|  0   |   1   | NNN AKLT     | (4/9)² ≈ 0.198 |        4        |

## Gate Construction

Create the projector gate for Protocol A:

In [3]:
# Spin projectors: P₀ (singlet), P₁ (triplet), P₂ (quintet)
P0 = total_spin_projector(0)
P1 = total_spin_projector(1)
P_not_2 = P0 + P1  # Projects out S=2 sector

# Protocol A gate: Coherent projection (preserves S=0/S=1 superposition)
proj_gate = SpinSectorProjection(P_not_2)

println("Gate constructed:")
println("  SpinSectorProjection(P₀+P₁) - coherent projection")
println()

Gate constructed:
  SpinSectorProjection(P₀+P₁) - coherent projection



## Protocol A: Coherent Projection with NN+NNN (SpinSectorProjection)

**How it works:**

Protocol A uses the `SpinSectorProjection` gate, which applies a projector operator to pairs of sites:

$$|\psi\rangle \to \frac{(P_0 + P_1)|\psi\rangle}{\|(P_0 + P_1)|\psi\rangle\|}$$

where $P_S$ projects onto total spin sector $S$. This:

1. **Removes S=2 quintets** from the two-qubit Hilbert space
2. **Preserves coherent superposition** of S=0 and S=1 sectors
3. **Does not collapse** the wavefunction stochastically

**NN+NNN Probabilistic Selection:**

We use `apply_with_prob!` to probabilistically choose between:
- **Nearest-Neighbor (NN)** projections with probability `p_nn = 0.7`: pairs (1,2), (3,4), ...
- **Next-Nearest-Neighbor (NNN)** projections with probability `1-p_nn = 0.3`: pairs (1,3), (5,7), ...

After L layers of mixed NN+NNN projections, this protocol still converges to the AKLT ground state, though possibly at a slower rate than pure NN (p_nn=1.0).

In [6]:
p_nn = 0.
L=16
n_layers = L 

16

In [7]:
println("═"^70)
println("Protocol A: SpinSectorProjection (Coherent) with NN+NNN")
println("═"^70)

# Define circuit using declarative API
# n_steps=1 means this circuit represents ONE layer
# simulate!(circuit, state; n_circuits=n_layers) runs it n_layers times
#
# IMPORTANT: For complete NNN coverage, we need all 4 sublayers:
# - :nnn_odd_1, :nnn_odd_2 (pairs (1,3), (5,7) and (3,5), (7,1))
# - :nnn_even_1, :nnn_even_2 (pairs (2,4), (6,8) and (4,6), (8,2))
# This covers all NNN pairs on a periodic chain.
circuit_A = Circuit(L=L, bc=bc, n_steps=1) do c
    # Single stochastic call: NN vs NNN with automatic pair coverage
    # :nn parity auto-expands to all NN pairs (combines :odd + :even)
    # :nnn parity auto-expands to all NNN pairs (combines 4 sublayers)
    apply_with_prob!(c; rng=:ctrl, outcomes=[
        (probability=p_nn, gate=proj_gate, geometry=Bricklayer(:nn)),
        (probability=1-p_nn, gate=proj_gate, geometry=Bricklayer(:nnn))
    ])
end

println("✓ Circuit defined with apply_with_prob! (p_nn=$p_nn)")
if p_nn == 1.0
    println("  - Pure NN: Bricklayer(:nn) auto-expands to all 12 NN pairs")
elseif p_nn == 0.0
    println("  - Pure NNN: Bricklayer(:nnn) auto-expands to all 12 NNN pairs")
else
    println("  - Mixed NN/NNN: p=$p_nn uses :nn, p=$(1-p_nn) uses :nnn")
end

# Initialize state with RNG for probabilistic decisions
rng_reg_A = RNGRegistry(ctrl=42, proj=1, haar=2, born=3)
state_A = SimulationState(L=L, bc=bc, site_type="S=1", maxdim=maxdim, rng=rng_reg_A)
state_A.mps = MPS(state_A.sites, ["Z0" for _ in 1:L])
println("✓ Initialized to |Z0⟩⊗$L (m=0 product state)")

# Track observables
track!(state_A, :entropy => EntanglementEntropy(cut=L÷2, order=1, base=2))
# Use order=2 for NNN regime (paired endpoints), order=1 for NN
so_order = p_nn == 1.0 ? 1 : (p_nn == 0.0 ? 2 : 1)
track!(state_A, :string_order => StringOrder(1, L÷2+1, order=so_order))
println("✓ Tracking: entropy, string_order")

# Run simulation: n_layers iterations of the circuit
println("\nRunning $n_layers layers of NN+NNN projections (p_nn=$p_nn)...")
simulate!(circuit_A, state_A; n_circuits=n_layers, record_when=:every_step)

# Report results
for layer in [1, n_layers÷2, n_layers]
    S = state_A.observables[:entropy][layer]
    SO = state_A.observables[:string_order][layer]
    println("  Layer $layer: S=$(round(S, digits=4)), |SO|=$(round(abs(SO), digits=4))")
end

S_final_A = state_A.observables[:entropy][end]
SO_final_A = state_A.observables[:string_order][end]

println("\nProtocol A Results:")
println("  Final entropy: $(round(S_final_A, digits=4))")
println("  Final |string order|: $(round(abs(SO_final_A), digits=4))")

# Expected physics based on p_nn
expected_SO = p_nn == 1.0 ? 4/9 : (p_nn == 0.0 ? (4/9)^2 : NaN)
expected_S = p_nn == 1.0 ? 2.0 : (p_nn == 0.0 ? 4.0 : NaN)

println("  Expected for p_nn=$p_nn: |SO| ≈ $(isnan(expected_SO) ? "mixed" : round(expected_SO, digits=3)), S ≈ $(isnan(expected_S) ? "mixed" : round(expected_S, digits=1))")

if !isnan(expected_SO) && !isnan(expected_S)
    if abs(abs(SO_final_A) - expected_SO) < 0.1 && abs(S_final_A - expected_S) < 1.0
        println("  ✅ CONVERGED to $(p_nn == 1.0 ? "NN" : "NNN") AKLT ground state!")
    else
        println("  ⚠️  Did not fully converge (try increasing n_layers)")
    end
else
    println("  ℹ️  Mixed NN/NNN regime - no simple analytical expectation")
end
println()

══════════════════════════════════════════════════════════════════════
Protocol A: SpinSectorProjection (Coherent) with NN+NNN
══════════════════════════════════════════════════════════════════════
✓ Circuit defined with apply_with_prob! (p_nn=0.0)
  - Pure NNN: Bricklayer(:nnn) auto-expands to all 12 NNN pairs
✓ Initialized to |Z0⟩⊗16 (m=0 product state)
✓ Tracking: entropy, string_order

Running 16 layers of NN+NNN projections (p_nn=0.0)...
  Layer 1: S=5.0055, |SO|=0.1554
  Layer 8: S=3.9973, |SO|=0.1979
  Layer 16: S=3.9973, |SO|=0.1979

Protocol A Results:
  Final entropy: 3.9973
  Final |string order|: 0.1979
  Expected for p_nn=0.0: |SO| ≈ 0.198, S ≈ 4.0
  ✅ CONVERGED to NNN AKLT ground state!



### Circuit Visualization

Visualize the circuit structure before running the simulation:

In [ ]:
# ASCII visualization (no dependencies)
print_circuit(circuit_A)

# SVG export (requires Luxor.jl)
# using Luxor
# plot_circuit(circuit_A; seed=42, filename="aklt_circuit.svg")